# Import thư viện và load dữ liệu

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import string
import time
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
from sklearn.metrics import confusion_matrix,f1_score, precision_score,recall_score,classification_report
from google.colab import drive
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy.sparse import csr_matrix

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# df = pd.read_csv("https://media.githubusercontent.com/media/PTIT-Projects/ttcs-svm-spam-email/refs/heads/main/dataset/combined_data.csv")
df = pd.read_csv("https://media.githubusercontent.com/media/PTIT-Projects/ttcs-svm-spam-email/refs/heads/main/dataset/sampled_dataset1000.csv")

# Tiền xử lý dữ liệu


In [ ]:

# viet thuong
df['text'] = df['text'].str.lower()

# xoa ky tu khong phai ASCII
df['text'] = df['text'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x) if isinstance(x, str) else x)

# xoa khoang trang
df['text'] = df['text'].apply(lambda x: re.sub(r'^\s+|\s+$', '', x).strip() if isinstance(x, str) else x)

# xoa html, xml
def remove_html_xml(text):
    try:
        soup = BeautifulSoup(text, 'html.parser')
        return soup.get_text()
    except:
        return text

df['text'] = df['text'].apply(remove_html_xml)

# xoa ky tu dac biet
def remove_special_characters(word):
    return word.translate(str.maketrans('', '', string.punctuation))

df['text'] = df['text'].apply(remove_special_characters)

# xoa url
def remove_urls(text):
    return re.sub(r'http\S+|www\S+|\S+\.(com|net|org|edu|gov|mil|int|info|biz|co)\S+', '', text)

df['text'] = df['text'].apply(remove_urls)

# xoa dia chi email
def remove_emails(text):
    return re.sub(r'\S+@\S+', '', text)

df['text'] = df['text'].apply(remove_emails)

# tach thanh cac tu
df['text'] = df['text'].apply(word_tokenize)

# xoa tu dung(tieng anh)
ENGLISH_STOP_WORDS = set(stopwords.words('english'))

def remove_stop_words(words):
    return [word for word in words if word not in ENGLISH_STOP_WORDS]

df['text'] = df['text'].apply(remove_stop_words)

# cat goc tu
stemmer = PorterStemmer()

def stem_words(words):
    return [stemmer.stem(word) for word in words]

df['text'] = df['text'].apply(stem_words)

# noi cac tu thanh chuoi
df['text'] = df['text'].apply(' '.join)

# dataset trainning voi test
X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# tfidf
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)





In [ ]:
# hashing_vectorizer
hashing_vectorizer = HashingVectorizer(n_features=3000)
X_train_hashed = hashing_vectorizer.fit_transform(X_train)
X_test_hashed = hashing_vectorizer.transform(X_test)


In [ ]:
print(X_train_hashed.shape)

(800, 3000)


# SVM

## Linear Hard Margin SVM

In [ ]:
class HardMarginSVM:
    """
    Optimized Hard Margin SVM implementation using gradient descent

    Attributes
    -------------
    eta : float
        Learning rate
    epoch : int
        Number of epochs
    random_state : int
        Random seed
    is_trained : bool
        Training completion flag
    num_samples : int
        Number of training samples
    num_features : int
        Number of features
    w : NDArray[float]
        Parameter vector: (num_features, ) ndarray
    b : float
        Bias parameter
    alpha : NDArray[float]
        Lagrange multipliers: (num_samples, ) ndarray
    """
    def __init__(self, eta=0.001, epoch=1000, random_state=42):
        self.eta = eta
        self.epoch = epoch
        self.random_state = random_state
        self.is_trained = False
        self.support_vectors = None

    def fit(self, X, y):
        """
        Fit parameter vector to training data

        Parameters
        --------------
        X : NDArray[NDArray[float]]
            Training data: (num_samples, num_features) matrix
        y : NDArray[float]
            Training labels: (num_samples) ndarray
        """
        # Convert sparse matrix to dense if needed
        if hasattr(X, "toarray"):
            X = X.toarray()

        self.num_samples = X.shape[0]
        self.num_features = X.shape[1]

        y_unique = np.unique(y)
        if len(y_unique) != 2:
            raise ValueError("Binary classification requires exactly 2 classes")

        if set(y_unique) == {0, 1}:
            y = np.where(y == 0, -1, 1)

        self.w = np.zeros(self.num_features)
        self.b = 0


        rgen = np.random.RandomState(self.random_state)
        self.alpha = rgen.uniform(low=0.0, high=0.01, size=self.num_samples)
        for i in range(self.epoch):
            self._cycle(X, y)

        sv_indices = np.where(self.alpha != 0)[0]

        self.support_vectors = sv_indices

        self.w = np.zeros(self.num_features)
        for i in sv_indices:
            self.w += self.alpha[i] * y[i] * X[i]

        bias_sum = 0
        for i in sv_indices:
            bias_sum += y[i] - np.dot(self.w, X[i])

        self.b = bias_sum / len(sv_indices)

        self.is_trained = True
        return self

    def predict(self, X):
        """
        Return predictions

        Parameters
        --------------
        X : NDArray[NDArray[float]]
            Data to classify: (any, num_features) matrix

        Returns
        ----------
        result : NDArray[int]
            Classification results 0 or 1: (any, ) ndarray
        """
        if not self.is_trained:
            raise Exception('Model not trained yet')

        # Convert sparse matrix to dense if needed
        if hasattr(X, "toarray"):
            X = X.toarray()

        decision_values = X @ self.w + self.b

        result = np.where(decision_values >= 0, 1, 0)
        return result

    def _cycle(self, X, y):
        """
        One gradient descent cycle

        Parameters
        --------------
        X : NDArray[NDArray[float]]
            Training data: (num_samples, num_features) matrix
        y : NDArray[float]
            Training labels: (num_samples) ndarray
        """
        y = y.reshape([-1, 1]) #(n,1)

        XXT = X @ X.T
        H = (y @ y.T) * XXT

        grad = np.ones(self.num_samples) - H @ self.alpha

        self.alpha += self.eta * grad

        self.alpha = np.clip(self.alpha, 0, None)


## Linear Hard Margin SVM cải tiến V1


In [ ]:
class HardMarginSVMV1:
    """
    Optimized Hard Margin SVM implementation using gradient descent

    Attributes
    -------------
    eta : float
        Learning rate
    epoch : int
        Number of epochs
    random_state : int
        Random seed
    is_trained : bool
        Training completion flag
    num_samples : int
        Number of training samples
    num_features : int
        Number of features
    w : NDArray[float]
        Parameter vector: (num_features, ) ndarray
    b : float
        Bias parameter
    alpha : NDArray[float]
        Lagrange multipliers: (num_samples, ) ndarray
    """
    def __init__(self, eta=0.001, epoch=1000, random_state=42, convergence_tol=1e-4):
        self.eta = eta
        self.epoch = epoch
        self.random_state = random_state
        self.convergence_tol = convergence_tol
        self.is_trained = False
        self.support_vectors = None

    def fit(self, X, y):
        """
        Fit parameter vector to training data

        Parameters
        --------------
        X : NDArray[NDArray[float]]
            Training data: (num_samples, num_features) matrix
        y : NDArray[float]
            Training labels: (num_samples) ndarray
        """
        # Convert sparse matrix to dense if needed
        if hasattr(X, "toarray"):
            X = X.toarray()
        self.num_samples = X.shape[0]
        self.num_features = X.shape[1]

        y_unique = np.unique(y)
        if len(y_unique) != 2:
            raise ValueError("Binary classification requires exactly 2 classes")

        if set(y_unique) == {0, 1}:
            y = np.where(y == 0, -1, 1)

        self.w = np.zeros(self.num_features)
        self.b = 0


        rgen = np.random.RandomState(self.random_state)
        self.alpha = rgen.uniform(low=0.0, high=0.01, size=self.num_samples)
        prev_alpha = np.zeros(self.num_samples)
        for i in range(self.epoch):
            np.copyto(prev_alpha, self.alpha)

            self._cycle(X, y)

            if i % 10 == 0:
                delta = np.linalg.norm(self.alpha - prev_alpha)
                if delta < self.convergence_tol:
                    break

        sv_indices = np.where(self.alpha != 0)[0]

        self.support_vectors = sv_indices

        self.w = np.zeros(self.num_features)
        for i in sv_indices:
            self.w += self.alpha[i] * y[i] * X[i]

        bias_sum = 0
        for i in sv_indices:
            bias_sum += y[i] - np.dot(self.w, X[i])

        self.b = bias_sum / len(sv_indices)

        self.is_trained = True
        return self

    def predict(self, X):
        """
        Return predictions

        Parameters
        --------------
        X : NDArray[NDArray[float]]
            Data to classify: (any, num_features) matrix

        Returns
        ----------
        result : NDArray[int]
            Classification results 0 or 1: (any, ) ndarray
        """
        if not self.is_trained:
            raise Exception('Model not trained yet')
        # Convert sparse matrix to dense if needed
        if hasattr(X, "toarray"):
            X = X.toarray()

        decision_values = X @ self.w + self.b

        result = np.where(decision_values >= 0, 1, 0)
        return result

    def _cycle(self, X, y):
        """
        One gradient descent cycle

        Parameters
        --------------
        X : NDArray[NDArray[float]]
            Training data: (num_samples, num_features) matrix
        y : NDArray[float]
            Training labels: (num_samples) ndarray
        """
        y = y.reshape([-1, 1])

        XXT = X @ X.T
        H = (y @ y.T) * XXT

        grad = np.ones(self.num_samples) - H @ self.alpha

        self.alpha += self.eta * grad

        self.alpha = np.clip(self.alpha, 0, None)

## Linear Soft Margin SVM

In [ ]:
import numpy as np

class LinearSVM:
    def __init__(self, C=1.0, max_iter=1000, lr=0.001, tolerance=1e-5):
        self.C = C
        self.max_iter = max_iter
        self.lr = lr
        self.tolerance = tolerance
        self.w = None
        self.b = 0

    def fit(self, X, y):
        # Convert labels to -1, 1 if they're 0, 1
        y_binary = np.where(y <= 0, -1, 1)

        n_samples, n_features = X.shape

        self.w = np.zeros(n_features)

        alpha = np.zeros(n_samples)

        # Pre-compute Gram matrix to avoid recalculation in the loop
        # K[i,j] = y_i * y_j * (x_i · x_j)
        K = np.dot(X, X.T) * np.outer(y_binary, y_binary)

        # SGD optimization
        for iteration in range(self.max_iter):
            alpha_prev = alpha.copy()

            # Vectorized margin calculation
            margins = 1 - K.dot(alpha)

            # Update all alphas in one step
            mask = margins > 0
            alpha[mask] += self.lr * margins[mask]

            # Apply box constraint
            alpha = np.clip(alpha, 0, self.C)

            # Check convergence
            if np.max(np.abs(alpha - alpha_prev)) < self.tolerance:
                break

        # Calculate weights
        self.w = np.dot(X.T, alpha * y_binary)

        # Calculate bias using support vectors
        sv_indices = alpha > 1e-5
        if np.any(sv_indices):
            self.b = np.mean(y_binary[sv_indices] - np.dot(X[sv_indices], self.w))

    def predict(self, X):
        """Predict class labels for samples in X."""
        return np.where(np.dot(X, self.w) + self.b >= 0, 1, 0)

    def get_parameters(self):
      print(f'w: {self.w}')
      print(f'b: {self.b}')

    def decision_function(self, X):
        """Return distance of samples to the decision boundary."""
        return np.dot(X, self.w) + self.b

## SVM tối ưu sử dụng thuật toán Pegasos

In [ ]:
class SVM:
    def __init__(self, lambda_param=1e-4, epoch=1000, batch_size=256, tol=1e-4, random_state=42):
        self.lambda_param = lambda_param
        self.epoch = epoch
        self.batch_size = batch_size
        self.tol = tol
        self.random_state = random_state
        self.is_trained = False

    def fit(self, X, y):
        if hasattr(X, "toarray"):
            X = csr_matrix(X)

        self.num_samples, self.num_features = X.shape

        y_unique = np.unique(y)
        if len(y_unique) != 2:
            raise ValueError("Phân loại nhị phân cần 2 nhãn")
        if set(y_unique) == {0, 1}:
            y = np.where(y == 0, -1, 1)

        self.w = np.zeros(self.num_features, dtype=np.float32)
        self.b = 0.0

        np.random.seed(self.random_state)
        t = 0
        previous_objective = float("inf")

        for ep in range(1, self.epoch + 1):
            indices = np.random.permutation(self.num_samples)
            for start in range(0, self.num_samples, self.batch_size):
                t += 1
                end = start + self.batch_size
                batch_idx = indices[start:end]
                X_batch = X[batch_idx]
                y_batch = y[batch_idx]

                eta = 1.0 / (self.lambda_param * t)
                margins = y_batch * (X_batch.dot(self.w) + self.b)
                mask = margins < 1
                self.w *= (1 - eta * self.lambda_param)
                if np.any(mask):
                    X_violate = X_batch[mask]
                    y_violate = y_batch[mask]
                    self.w += (eta / self.batch_size) * np.dot(y_violate, X_violate.toarray() if hasattr(X_violate, "toarray") else X_violate)
                    self.b += (eta / self.batch_size) * np.sum(y_violate)
                norm_w = np.linalg.norm(self.w)
                factor = min(1, (1.0 / np.sqrt(self.lambda_param)) / (norm_w))
                self.w *= factor

            decision = X.dot(self.w) + self.b
            hinge_losses = np.maximum(0, 1 - y * decision)
            objective = 0.5 * self.lambda_param * np.dot(self.w, self.w) + np.mean(hinge_losses)

            if ep % 10 == 0:
                print(f"Epoch {ep}, Giá trị hàm mục tiêu: {objective:.4f}")

            if abs(previous_objective - objective) < self.tol:
                print(f"Dừng sớm tại epoch {ep}, giá trị hàm mục tiêu thay đổi: {abs(previous_objective - objective):.6f}")
                break
            previous_objective = objective

        self.is_trained = True
        return self

    def predict(self, X):
        if not self.is_trained:
            raise Exception("Mô hình chưa được huấn luỵen")

        if hasattr(X, "toarray"):
            X = csr_matrix(X)

        decision = X.dot(self.w) + self.b
        return np.where(decision >= 0, 1, 0)

# Train model và dự đoán

In [ ]:
X_train_dense = X_train_tfidf.toarray()
X_test_dense = X_test_tfidf.toarray()



## Linear SVM Hard Margin ban đầu

In [ ]:
hard_margin_svm = HardMarginSVM()
hard_margin_svm.fit(X_train_dense, y_train.to_numpy())

In [ ]:
y_pred = hard_margin_svm.predict(X_test_dense)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.93      0.94        98
           1       0.93      0.96      0.95       102

    accuracy                           0.94       200
   macro avg       0.95      0.94      0.94       200
weighted avg       0.95      0.94      0.94       200



In [ ]:
print(confusion_matrix(y_test, y_pred))

[[91  7]
 [ 4 98]]


### Thử với hashing vectorizer


In [ ]:
linear_svm = HardMarginSVM()
linear_svm.fit(X_train_hashed, y_train.to_numpy())


In [ ]:
y_pred = linear_svm.predict(X_test_hashed)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.88      0.90        98
           1       0.89      0.93      0.91       102

    accuracy                           0.91       200
   macro avg       0.91      0.90      0.90       200
weighted avg       0.91      0.91      0.90       200

[[86 12]
 [ 7 95]]


## Linear SVM Hard Margin cải tiến V1

In [ ]:
hard_margin_svm_v1 = HardMarginSVMV1()
hard_margin_svm_v1.fit(X_train_dense, y_train.to_numpy())

KeyboardInterrupt: 

In [ ]:
y_pred = hard_margin_svm_v1.predict(X_test_dense)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

## Linear SVM Soft Margin

In [ ]:
linear_svm = LinearSVM()
linear_svm.fit(X_train_dense, y_train.to_numpy())
y_pred = linear_svm.predict(X_test_dense)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
print(linear_svm.get_parameters())

In [ ]:
linear_svm = LinearSVM()
linear_svm.fit(X_train_hashed, y_train.to_numpy())
y_pred = linear_svm.predict(X_test_hashed)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Áp dụng feature extraction và so sánh

## Khi không sử dụng feature extraction

In [ ]:

pegasos = SVM()

start = time.time()
pegasos.fit(X_train_tfidf, y_train.to_numpy())
end = time.time()

y_pred_pegasos = pegasos.predict(X_test_tfidf)


print("Thời gian huấn luyện (Pegasos):", end - start, "giây")
print("Accuracy (Pegasos):", accuracy_score(y_test, y_pred_pegasos))
print("F1-score (Pegasos):", f1_score(y_test, y_pred_pegasos))

hard_svm = HardMarginSVMV1()

start = time.time()
hard_svm.fit(X_train_tfidf, y_train)
end = time.time()

y_pred_hard = hard_svm.predict(X_test_tfidf)

print("Thời gian huấn luyện (Hard-margin v1):", end - start, "giây")
print("Accuracy (Hard-margin v1):", accuracy_score(y_test, y_pred_hard))
print("F1-score (Hard-margin v1):", f1_score(y_test, y_pred_hard))



Epoch 10, Giá trị hàm mục tiêu: 32.6296
Epoch 20, Giá trị hàm mục tiêu: 1.1482
Epoch 30, Giá trị hàm mục tiêu: 1.6400
Epoch 40, Giá trị hàm mục tiêu: 1.4125
Epoch 50, Giá trị hàm mục tiêu: 0.4692
Epoch 60, Giá trị hàm mục tiêu: 0.3155
Epoch 70, Giá trị hàm mục tiêu: 0.2233
Epoch 80, Giá trị hàm mục tiêu: 0.2225
Epoch 90, Giá trị hàm mục tiêu: 0.1400
Epoch 100, Giá trị hàm mục tiêu: 0.6319
Epoch 110, Giá trị hàm mục tiêu: 0.1472
Epoch 120, Giá trị hàm mục tiêu: 0.1190
Epoch 130, Giá trị hàm mục tiêu: 0.1022
Epoch 140, Giá trị hàm mục tiêu: 0.0883
Epoch 150, Giá trị hàm mục tiêu: 0.0779
Epoch 160, Giá trị hàm mục tiêu: 0.1303
Epoch 170, Giá trị hàm mục tiêu: 0.0809
Epoch 180, Giá trị hàm mục tiêu: 0.0704
Epoch 190, Giá trị hàm mục tiêu: 0.1427
Epoch 200, Giá trị hàm mục tiêu: 0.0667
Epoch 210, Giá trị hàm mục tiêu: 0.0611
Epoch 220, Giá trị hàm mục tiêu: 0.0556
Epoch 230, Giá trị hàm mục tiêu: 0.5965
Epoch 240, Giá trị hàm mục tiêu: 0.0570
Epoch 250, Giá trị hàm mục tiêu: 0.0526
Epoch 26

## Khi sử dụng feature extraction

### k = 5000

In [ ]:
selector = SelectKBest(score_func  = chi2, k =5000)
selector.fit(X_train_tfidf, y_train)
X_train_reduced = selector.transform(X_train_tfidf)
X_test_reduced = selector.transform(X_test_tfidf)
print(f'X_train_reduced shape: {X_train_reduced.shape}')
print(f'X_train_tfidf shape: {X_train_tfidf.shape}')

X_train_reduced shape: (800, 5000)
X_train_tfidf shape: (800, 15517)


In [ ]:

pegasos = SVM()

start = time.time()
pegasos.fit(X_train_reduced, y_train.to_numpy())
end = time.time()

y_pred_pegasos = pegasos.predict(X_test_reduced)


print("Thời gian huấn luyện (Pegasos):", end - start, "giây")
print("Accuracy (Pegasos):", accuracy_score(y_test, y_pred_pegasos))
print("F1-score (Pegasos):", f1_score(y_test, y_pred_pegasos))

hard_svm = HardMarginSVMV1()

start = time.time()
hard_svm.fit(X_train_reduced, y_train)
end = time.time()

y_pred_hard = hard_svm.predict(X_test_reduced)

print("Thời gian huấn luyện (Hard-margin v1):", end - start, "giây")
print("Accuracy (Hard-margin v1):", accuracy_score(y_test, y_pred_hard))
print("F1-score (Hard-margin v1):", f1_score(y_test, y_pred_hard))



Epoch 10, Giá trị hàm mục tiêu: 15.4006
Epoch 20, Giá trị hàm mục tiêu: 0.7687
Epoch 30, Giá trị hàm mục tiêu: 6.2444
Epoch 40, Giá trị hàm mục tiêu: 4.8203
Epoch 50, Giá trị hàm mục tiêu: 0.4487
Epoch 60, Giá trị hàm mục tiêu: 0.2938
Epoch 70, Giá trị hàm mục tiêu: 0.3182
Epoch 80, Giá trị hàm mục tiêu: 0.2201
Epoch 90, Giá trị hàm mục tiêu: 0.1757
Epoch 100, Giá trị hàm mục tiêu: 0.1468
Epoch 110, Giá trị hàm mục tiêu: 0.1228
Epoch 120, Giá trị hàm mục tiêu: 0.3925
Epoch 130, Giá trị hàm mục tiêu: 0.1167
Epoch 140, Giá trị hàm mục tiêu: 0.1003
Epoch 150, Giá trị hàm mục tiêu: 0.1120
Epoch 160, Giá trị hàm mục tiêu: 0.0943
Epoch 170, Giá trị hàm mục tiêu: 0.0937
Epoch 180, Giá trị hàm mục tiêu: 0.0800
Epoch 190, Giá trị hàm mục tiêu: 0.0727
Epoch 200, Giá trị hàm mục tiêu: 0.0762
Epoch 210, Giá trị hàm mục tiêu: 0.0694
Dừng sớm tại epoch 210, giá trị hàm mục tiêu thay đổi: 0.000055
Thời gian huấn luyện (Pegasos): 1.9836857318878174 giây
Accuracy (Pegasos): 0.925
F1-score (Pegasos): 0.

### k = 10000

In [ ]:
selector = SelectKBest(score_func  = chi2, k =10000)
selector.fit(X_train_tfidf, y_train)
X_train_reduced = selector.transform(X_train_tfidf)
X_test_reduced = selector.transform(X_test_tfidf)
print(f'X_train_reduced shape: {X_train_reduced.shape}')
print(f'X_train_tfidf shape: {X_train_tfidf.shape}')

X_train_reduced shape: (800, 10000)
X_train_tfidf shape: (800, 15517)


In [ ]:

pegasos = SVM()

start = time.time()
pegasos.fit(X_train_reduced, y_train.to_numpy())
end = time.time()

y_pred_pegasos = pegasos.predict(X_test_reduced)


print("Thời gian huấn luyện (Pegasos):", end - start, "giây")
print("Accuracy (Pegasos):", accuracy_score(y_test, y_pred_pegasos))
print("F1-score (Pegasos):", f1_score(y_test, y_pred_pegasos))

hard_svm = HardMarginSVMV1()

start = time.time()
hard_svm.fit(X_train_reduced, y_train)
end = time.time()

y_pred_hard = hard_svm.predict(X_test_reduced)

print("Thời gian huấn luyện (Hard-margin v1):", end - start, "giây")
print("Accuracy (Hard-margin v1):", accuracy_score(y_test, y_pred_hard))
print("F1-score (Hard-margin v1):", f1_score(y_test, y_pred_hard))



Epoch 10, Giá trị hàm mục tiêu: 17.9088
Epoch 20, Giá trị hàm mục tiêu: 1.4727
Epoch 30, Giá trị hàm mục tiêu: 6.8120
Epoch 40, Giá trị hàm mục tiêu: 3.4568
Epoch 50, Giá trị hàm mục tiêu: 0.4446
Epoch 60, Giá trị hàm mục tiêu: 0.2895
Epoch 70, Giá trị hàm mục tiêu: 0.2126
Epoch 80, Giá trị hàm mục tiêu: 1.1532
Epoch 90, Giá trị hàm mục tiêu: 0.1964
Epoch 100, Giá trị hàm mục tiêu: 0.1579
Epoch 110, Giá trị hàm mục tiêu: 0.1328
Epoch 120, Giá trị hàm mục tiêu: 0.1307
Epoch 130, Giá trị hàm mục tiêu: 0.1110
Epoch 140, Giá trị hàm mục tiêu: 0.0958
Epoch 150, Giá trị hàm mục tiêu: 0.0850
Epoch 160, Giá trị hàm mục tiêu: 0.0746
Epoch 170, Giá trị hàm mục tiêu: 0.0893
Epoch 180, Giá trị hàm mục tiêu: 0.0756
Epoch 190, Giá trị hàm mục tiêu: 0.1257
Epoch 200, Giá trị hàm mục tiêu: 0.0685
Epoch 210, Giá trị hàm mục tiêu: 0.0786
Epoch 220, Giá trị hàm mục tiêu: 0.0582
Epoch 230, Giá trị hàm mục tiêu: 0.1271
Epoch 240, Giá trị hàm mục tiêu: 0.0636
Dừng sớm tại epoch 249, giá trị hàm mục tiêu tha

### Nhận xét
Khi sử dụng feature extraction thì số chiều của dữ liệu giảm, chạy nhanh hơn tuy nhiên thì độ chính xác cũng như f1 score giảm. Do bản thân thuật toán Pegasos cho SVM chạy tương đối nhanh và không tốn quá nhiều bộ nhớ nên nhóm tập trung vào độ chính xác và f1 score cho mô hình. Do đó nhóm em không áp dụng feature extraction cho bộ dữ liệu gốc